In [1]:
import autogen

In [2]:
from autogen import ConversableAgent,Agent

In [3]:
config_list = [
    {
        'model': 'gpt-4-1106-preview',
        'api_key': 'aitools',
        'api_base': 'http://aitools.cs.vt.edu:7860/openai/v1'
    },
    {
        'model': 'gpt-3.5-turbo-16k',
        'api_key': 'aitools',
        'api_base': 'http://aitools.cs.vt.edu:7860/openai/v1'
    },
]

llm_config = {
    "request_timeout": 300,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0.1,
    "allow_format_str_template": True}


In [60]:
import re
from typing import Optional, List, Dict, Any, Union, Tuple
import pandas as pd
import json

import autogen
import requests
from autogen import Agent, ConversableAgent
class DatasetScaperAgent(autogen.AssistantAgent):
    def __init__(self):
        super().__init__(name="DatasetScraperAgent", llm_config=None,
                         max_consecutive_auto_reply=3)
        self.register_reply([Agent, None], self._scraper_func)

    @staticmethod
    def _extract_urls(text):
        """
            Extracts a list of URLs from a given string.

            :param text: str, the text to search for URLs.
            :return: list, a list of URLs found in the string.
            """
        url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        urls = re.findall(url_regex, text)
        return urls
    
    def _download_dataset(self, url):
        result: Optional[Dict[str, str]] = dict()
        dataframe = pd.read_csv(url).iloc[:500]
        json_ = dataframe.to_json()
        result["content"] = json_
        return result




    def _scraper_func(self,
                      recipient: ConversableAgent,
                      messages: Optional[List[Dict]] = None,
                      sender: Optional[Agent] = None,
                      config: Optional[Any] = None,
                      ) -> Any | Union[str, Dict, None]:
        url = self._extract_urls(messages[0]["content"])
        description = self._download_dataset(url[0])
        if (description is None) or (len(description) == 0):
            return False, None
        return True, description

In [61]:
from typing import Optional

from autogen import ConversableAgent


class Blueprint:

    def __init__(self,
                 agents: Optional[list[ConversableAgent]] = None,
                 config_list: Optional[list[dict]] = None,
                 llm_config: Optional[dict] = None):
        self._agents = agents or None
        self._config_list = config_list or None
        self._llm_config = llm_config or None

        self._initiator = agents[0] if agents else None
        self._recipient = agents[1] if agents and len(agents) > 1 else None

    async def initiate_work(self, message: str):
        if self._initiator and self._recipient:
            self._initiator.initiate_chat(recipient=self._recipient, message=message)
        else:
            raise ValueError("No initiator or recipient agent found.")


In [62]:
import os
import sys
DEFAULT_MODEL = "gpt-4"
FAST_MODEL = "gpt-3.5-turbo"
# Regular expression for finding a code block
CODE_BLOCK_PATTERN = r"(.*?)```(\w*)\n(.*?)\n```"
WORKING_DIR = os.path.join(os.getcwd(), ".code")
UNKNOWN = "unknown"
TIMEOUT_MSG = "Timeout"
DEFAULT_TIMEOUT = 600
WIN32 = sys.platform == "win32"
PATH_SEPARATOR = WIN32 and "\\" or "/"

In [63]:
import autogen

In [64]:
import re
from typing import Optional, List, Dict, Any, Union, Tuple
import pandas as pd
import json

import autogen
import requests
from autogen import Agent, ConversableAgent

In [72]:
class DatasetScaperAgent(autogen.AssistantAgent):
    def __init__(self):
        super().__init__(name="DatasetScraperAgent", llm_config=None,
                         max_consecutive_auto_reply=3)
        self.register_reply([Agent, None], self._scraper_func)

    @staticmethod
    def _extract_urls(text):
        """
            Extracts a list of URLs from a given string.

            :param text: str, the text to search for URLs.
            :return: list, a list of URLs found in the string.
            """
        url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        urls = re.findall(url_regex, text)
        return urls
    
    def _download_dataset(self, url):
        result: Optional[Dict[str, str]] = dict()
        dataframe = pd.read_csv(url)
        json_ = dataframe.to_json()
        result["content"] = json_
        return result




    def _scraper_func(self,
                      recipient: ConversableAgent,
                      messages: Optional[List[Dict]] = None,
                      sender: Optional[Agent] = None,
                      config: Optional[Any] = None,
                      ) -> Any | Union[str, Dict, None]:
        url = self._extract_urls(messages[0]["content"])
        description = self._download_dataset(url[0])
        if (description is None) or (len(description) == 0):
            return False, None
        return True, description

In [66]:
from typing import Optional, Dict, List

from autogen import ConversableAgent, Agent


class CustomBlueprintML(Blueprint):
    def __init__(self, work_dir: Optional[str] = WORKING_DIR):
        super().__init__([], config_list=config_list, llm_config=llm_config)
        self._work_dir = work_dir or "code"
        self._summary_result: Optional[str] = None
    
    @property
    def summary_result(self) -> str | None:
        """The getter for the 'summary_result' attribute."""
        return self._summary_result
    
    @property
    def work_dir(self) -> str:
        """The getter for the 'work_dir' attribute."""
        return self._work_dir

    async def status(self) -> Dict[Agent, List[Dict[str, str]]]:
        return self._agent0.chat_messages
    
    async def initiate_work(self, message: str):
        clear_working_dir(self._work_dir)
        self.boss_agent = ConversableAgent("boss_agent", 
                                      max_consecutive_auto_reply=6,
                                      llm_config=False,
                                      human_input_mode="NEVER",
                                      code_execution_config=False,
                                      function_map=None,
                                      system_message="""You are a boss.  You have a team of data scientists.
                                      You have a dataset that you want to use to train a machine learning model.
                                      You want to know what the best model is for your dataset.
                                      You want to train your machine learning model on your dataset.
                                      """) 
        
        self.dataset_scaper_agent = DatasetScaperAgent()

        self.coder_agent = ConversableAgent("coder_agent",
                                            max_consecutive_auto_reply=12,
                                            llm_config=llm_config,
                                            human_input_mode="NEVER",
                                            code_execution_config=False,
                                            function_map=None,
                                            system_message="""You are a helpful AI assistant.
                                            You are a python coder and a machine learning expert.
                                            Lets use sklearn package for training our model.
                                            Use pandas for loading the dataset and preprocessing related tasks.  
                                            You have been provided with the dataset description and the dataset as a list i.e[dataset_description, dataframe_url].
                                            use the url to load the dataset into a dataframe and do the next steps.
                                            You know how to train a machine learning model.
                                            You have a dataset that you want to use to train a machine learning model.
                                            You need to preprocess the dataset,remove the missing values.
                                            You need to perform feature engineering on the dataset provided.
                                            You want to know what the best model is for your dataset.
                                            You want to train your machine learning model on your dataset.
                                            Do not suggest incomplete code which requires users to modify.
                                            Always put `# filename: /<filename>` as the first line of each code block.
                                            Feel free to include multiple code blocks in one response. Do not ask users to copy and paste the result.
                                            """
                                            )
        
        self.critic_agent = ConversableAgent("critic_agent",
                                             max_consecutive_auto_reply=12,
                                             llm_config=llm_config,
                                             human_input_mode="NEVER",
                                             code_execution_config=False,
                                             function_map=None,
                                             system_message=""" You are a QA developer expert in Python, using the pytest framework.
                                                You're writing code to solve a datascience project.
                                                When you receive a message, you should expect that message to describe the steps for data preprocessing and training a model code.
                                                Let's use sklearn for our training our model.
                                                You must check weather the feature engineering is done properly.
                                                You must check weather there is code for handling missing values.
                                                You must check weather the feature engineering is done properly.  
                                                You must check weather the model is trained and inference is done properly.
                                                You must indicate the script type in the code block.
                                                Check for the above mentioned things and give the feedback and suggest improvements that can be done.    
        """
                                             )
                                             

        self.boss_agent.initiate_chat(self.dataset_scaper_agent, True, True, message=message)
        message = self.boss_agent.last_message(self.dataset_scaper_agent)
        self.boss_agent.initiate_chat(self.coder_agent, True, True, message=message)
        coder_message = self.boss_agent.last_message(self.coder_agent)
        self.coder_agent.initiate_chat(self.critic_agent, True, True, message=coder_message)
        critic_message = self.coder_agent.last_message(self.critic_agent)
        self.coder_agent.initiate_chat(self.critic_agent, True, True, message=critic_message)
        coder_message = self.coder_agent.last_message(self.critic_agent)["content"]
        save_code_files(coder_message, self.work_dir)
        self._summary_result = summarize_files(self.work_dir)






        

In [73]:
task = f"""
            I want to retrieve the dataset from the url https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv
            build an end to end datascience project, use the dataset from the above link.
            You need to preprocess the dataset,remove the missing values.
            You need to perform feature engineering on the dataset provided.
            You want to know what the best model is for your dataset.
            You want to train your machine learning model on your dataset.
            """

In [74]:
from autogen import ConversableAgent, Agent

In [83]:
boss_agent = ConversableAgent("boss_agent", 
                                      max_consecutive_auto_reply=3,
                                      llm_config=False,
                                      human_input_mode="NEVER",
                                      code_execution_config=False,
                                      function_map=None,
                                      system_message="""You are a boss.  You have a team of data scientists.
                                      You have a dataset that you want to use to train a machine learning model.
                                      You want to know what the best model is for your dataset.
                                      You want to train your machine learning model on your dataset.
                                      """) 
        
dataset_scaper_agent = DatasetScaperAgent()

coder_agent = ConversableAgent("coder_agent",
                                max_consecutive_auto_reply=3,
                                llm_config=llm_config,
                                human_input_mode="NEVER",
                                code_execution_config=False,
                                function_map=None,
                                system_message="""You are a helpful AI assistant.
                                You are a python coder and a machine learning expert,you are building a datascience pipeline.
                                Write python code for the below requirements.
                                You have been provided with the url and dataset description from the other agent.
                                extract the url from the above and use it in the further steps
                                Lets use sklearn package for training our model.
                                Use pandas for loading the dataset and preprocessing related tasks.  
                                use the url to load the dataset into a dataframe and do the next steps.
                                You know how to train a machine learning model.
                                You have a dataset that you want to use to train a machine learning model.
                                You need to preprocess the dataset,remove the missing values.
                                You need to perform feature engineering on the dataset provided.
                                You want to know what the best model is for your dataset.
                                You want to train your machine learning model on your dataset.
                                Do not suggest incomplete code which requires users to modify.
                                Always put `# filename: /<filename>` as the first line of each code block.
                                Feel free to include multiple code blocks in one response. Do not ask users to copy and paste the result.
                                """
                                )
        
critic_agent = ConversableAgent("critic_agent",
                                max_consecutive_auto_reply=3,
                                llm_config=llm_config,
                                human_input_mode="NEVER",
                                code_execution_config=False,
                                function_map=None,
                                system_message=""" You are a QA developer expert in Python, using the pytest framework.
                                You're writing code to solve a datascience project.
                                When you receive a message, you should expect that message to describe the steps for data preprocessing and training a model code.
                                Let's use sklearn for our training our model.
                                You must check weather the feature engineering is done properly.
                                You must check weather there is code for handling missing values.
                                You must check weather the feature engineering is done properly.  
                                You must check weather the model is trained and inference is done properly.
                                You must indicate the script type in the code block.
                                Check for the above mentioned things and give the feedback and suggest improvements that can be done.
""")

In [84]:
task

'\n            I want to retrieve the dataset from the url https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv\n            build an end to end datascience project, use the dataset from the above link.\n            You need to preprocess the dataset,remove the missing values.\n            You need to perform feature engineering on the dataset provided.\n            You want to know what the best model is for your dataset.\n            You want to train your machine learning model on your dataset.\n            '

In [86]:
boss_agent.initiate_chat(dataset_scaper_agent, True, False, message=task)
message = boss_agent.last_message(dataset_scaper_agent)
print(message)
boss_agent.initiate_chat(coder_agent, True,False, message=message)
coder_message = boss_agent.last_message(coder_agent)
print(coder_message)
coder_agent.initiate_chat(critic_agent, True, False, message=coder_message)
critic_message = coder_agent.last_message(critic_agent)
print(critic_message)
critic_agent.initiate_chat(coder_agent, True, False, message=critic_message)
coder_message = critic_agent.last_message(coder_agent)
print(coder_message)

boss_agent (to DatasetScraperAgent):


            I want to retrieve the dataset from the url https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv
            build an end to end datascience project, use the dataset from the above link.
            You need to preprocess the dataset,remove the missing values.
            You need to perform feature engineering on the dataset provided.
            You want to know what the best model is for your dataset.
            You want to train your machine learning model on your dataset.
            

--------------------------------------------------------------------------------
DatasetScraperAgent (to boss_agent):

{"sepal_length":{"0":5.1,"1":4.9,"2":4.7,"3":4.6,"4":5.0,"5":5.4,"6":4.6,"7":5.0,"8":4.4,"9":4.9,"10":5.4,"11":4.8,"12":4.8,"13":4.3,"14":5.8,"15":5.7,"16":5.4,"17":5.1,"18":5.7,"19":5.1,"20":5.4,"21":5.1,"22":4.6,"23":5.1,"24":4.8,"25":5.0,"26":5.0,"27":5.2,"28":5.2,"

In [87]:
#
# This file contains the code for the `utils.py` module.
#
# The idea here is that we do not want to execute code, but we want to generate code.
# This is currently based on the issue that we cannot execute code in the browser
# because the signal library is not available on a background thread in a streamlit process.
#
# So this is a workaround to generate code instead of executing it, and still depends on
# the autogen library `code_utils`.
#
import ast
import glob
import json
import os
import re
import textwrap
from hashlib import md5
from typing import List
from typing import Optional
from typing import Tuple

from autogen import Agent, ConversableAgent



def extract_code(
        text: str, pattern: str = CODE_BLOCK_PATTERN,
) -> List[Tuple[str, str]]:
    """
    Extract code blocks from a text and format preambles as comments.

    This function searches for code blocks in a string of text. Each code block is expected
    to be in the format of an optional preamble followed by the code enclosed in triple
    backticks. The language of the code can be optionally specified right after the first
    set of triple backticks.

    The function returns a list of 2-tuples, each containing the language (if specified,
    otherwise an empty string) and the code. If no code blocks are detected, the function
    returns a list with a single 2-tuple containing the string "unknown" and the original text.

    Preambles are treated as follows:
    - Each line of the preamble is turned into a comment in the code block by prefixing it
      with a '#' character, unless it already starts with '#'.
    - If a line in the preamble starts with '# filename', this line is not commented out,
      and it is placed as the first line of the corresponding code block. This is meant
      to preserve a directive that might be important for the interpretation or execution
      of the code. Only the first occurrence of a '# filename' line is processed in this manner.

    Args:
        text (str): The text to extract code from.
        pattern (str, optional): The regular expression pattern for finding the
            code block. Defaults to CODE_BLOCK_PATTERN.

    Returns:
        list: A list of tuples, each containing the language and the code (with the preamble
              incorporated as comments and the '# filename' line placed first if present).
    """
    matches = re.findall(pattern, text, flags=re.DOTALL)

    result = []

    for match in matches:
        preamble, lang, code = match
        preamble_filename_line = ""
        code_filename_line = ""
        preamble_lines = []
        code_lines = []

        # Process each line in the preamble
        for line in preamble.split("\n"):
            line_stripped = line.strip()
            if line_stripped.startswith("# filename"):
                # Capture the filename line to place at the beginning of the code block
                if not preamble_filename_line:  # Only capture the first occurrence
                    preamble_filename_line = line_stripped
            else:
                # Add '#' to each line of the preamble if it doesn't start with '#'
                if line_stripped and not line_stripped.startswith("#"):
                    preamble_lines.append(f"# {line}")
                else:
                    preamble_lines.append(line)

        # Concatenate the processed preamble lines with newlines
        commented_preamble = "\n".join(preamble_lines)

        # Also look for filename inside the code block
        for line in code.split("\n"):
            line_stripped = line.strip()
            if line_stripped.startswith("# filename"):
                # Capture the filename line to place at the beginning of the code block
                if not code_filename_line:
                    code_filename_line = line_stripped
            else:
                code_lines.append(line)
        full_code_lines = "\n".join(code_lines)

        filename_line = code_filename_line or preamble_filename_line or ""
        # Concatenate the filename line (if any), the commented preamble, and the code, with newlines in between
        full_code = "\n".join(filter(None, [filename_line, commented_preamble, full_code_lines]))

        result.append((lang, full_code))

    if not result:
        result.append((UNKNOWN, text))

    return result


def _get_function_signature(function: ast.FunctionDef) -> str:
    """
    Construct the function signature from the function definition, including type annotations.

    Args:
        function (ast.FunctionDef): The function definition.

    Returns:
        str: The function signature as a string.
    """
    # Extract the function name
    func_name = function.name

    # Extract the function arguments
    args = []
    for arg in function.args.args:
        arg_name = arg.arg

        # Include type annotation if present
        if arg.annotation:
            arg_name += f": {ast.unparse(arg.annotation)}"

        args.append(arg_name)

    # Handle default values for arguments if they exist
    defaults_offset = len(args) - len(function.args.defaults)
    for i, default in enumerate(function.args.defaults):
        args[i + defaults_offset] += f"={ast.unparse(default)}"

    # Handle type annotations for return value
    return_annotation = ""
    if function.returns:
        return_annotation = f" -> {ast.unparse(function.returns)}"

    args_str = ", ".join(args)
    return f"{func_name}({args_str}){return_annotation}"


def _get_public_functions(file_path: str):
    """
    Extract the public functions and their signatures from a Python file, including those inside classes.

    Args:
        file_path (str): The path to the Python file.

    Returns:
        list: A list of strings, each representing a public function or method with its signature.
    """
    with open(file_path, "r", encoding='utf-8') as source:
        node = ast.parse(source.read())

    functions = []
    for item in node.body:
        if isinstance(item, ast.FunctionDef):
            # For top-level functions
            if not item.name.startswith("_"):
                functions.append(_get_function_signature(item))
        elif isinstance(item, ast.ClassDef):
            # For functions within classes (methods)
            for class_item in item.body:
                if (isinstance(class_item, ast.FunctionDef) or isinstance(class_item, ast.AsyncFunctionDef)) and not class_item.name.startswith("_"):
                    functions.append(f"{item.name}.{_get_function_signature(class_item)}")  # Include the class name

    return functions


def summarize_files(working_folder: str) -> str:
    """
    Summarizes each Python file's public interface in the specified folder and its subfolders.

    Args:
        working_folder (str): The path of the working folder.

    Returns:
        str: A markdown string summarizing the public interface of each Python file.
    """
    markdown_summary = ""

    for subdir, _, files in os.walk(working_folder):
        for file in files:
            if file.endswith(".py"):
                file_path = os.path.join(subdir, file)
                public_functions = _get_public_functions(file_path)

                # Create a relative file path to display
                relative_file_path = os.path.relpath(file_path, working_folder)

                # Add a section for this file to the markdown summary
                markdown_summary += f"## {relative_file_path}\n"

                if public_functions:
                    for signature in public_functions:
                        markdown_summary += f"- `{signature}`\n"
                else:
                    markdown_summary += "No public methods.\n"

                # Add an extra newline to separate sections
                markdown_summary += "\n"

    return markdown_summary


def save_code(
        code: str = None,
        work_dir: Optional[str] = None,
        lang: Optional[str] = "python",
) -> str:
    """
    Save the code in a working directory.
    Filenames are expected to be relative paths, and appear inside the code
    as a line starting with '# filename'.

    Args:
        code (str): The code to save.  If a filename cannot be derived from a line starting with '# filename',
            a file with a randomly generated name will be created.
            The file name must be a relative path. Relative paths are relative to the working directory.
        work_dir (Optional, str): The working directory for the code execution.
            If None, a default working directory will be used.
            The default working directory is the "code" directory under the path to this file.
            If the code is executed in the current environment, the code must be trusted.
        lang (Optional, str): The language of the code. Default is "python".

    Returns:
        str: A summary of the files in the working directory
        image: The docker image name after container run when docker is used.
    """
    filename = re.findall(r"# filename: ([^\n]+)", code)
    if not filename:
        code_hash = md5(code.encode()).hexdigest()
        # create a file with a automatically generated name
        filename = f"tmp_code_{code_hash}.{'py' if lang.startswith('python') else lang}"
    elif isinstance(filename, list):
        filename = filename[0]
    work_dir = work_dir or WORKING_DIR
    filepath = os.path.join(work_dir, filename)
    file_dir = os.path.dirname(filepath)
    print(f"Saving code to {filepath}")
    os.makedirs(file_dir, exist_ok=True)
    if code is not None:
        with open(filepath, "w", encoding="utf-8") as fout:
            fout.write(code)
    return filename

def save_code_files(llm_message: str, work_dir: Optional[str] = None) -> List[str]:
    filenames = []
    code_files = extract_code(llm_message)
    for lang, code_block in code_files:
        if lang == 'python':
            filename = save_code(code_block, work_dir)
            filenames.append(filename)
    return filenames
def clear_working_dir(work_dir: Optional[str] = None, filename_wildcard: str = '*.py') -> None:
    """
    Clears all .py files from the specified working directory.
    If no directory is specified, the WORKING_DIR constant is used.

    After removing a .py file, if no other files are present in the directory,
    it removes the directory itself.

    Args:
        work_dir (Optional[str], optional): The directory to clear .py files from. Defaults to None.

    Returns:
        None
    """
    work_dir = work_dir or WORKING_DIR  # If work_dir is None, use WORKING_DIR

    if os.path.exists(work_dir) and os.path.isdir(work_dir):
        # Find all .py files in the working directory and subdirectories
        python_files = glob.glob(os.path.join(work_dir, '**', filename_wildcard), recursive=True)

        for file_path in python_files:
            try:
                os.remove(file_path)  # Delete the file
                print(f"Deleted {file_path}")  # Optional: printing confirmation

                # Get the directory where the file was located
                dir_path = os.path.dirname(file_path)

                # Check if the directory is empty
                if not os.listdir(dir_path):
                    # Remove the empty directory
                    os.removedirs(dir_path)
                    print(f"Removed directory {dir_path}")

            except Exception as e:
                print(f"Could not delete {file_path} due to {e}")  # Handle exceptions during file or directory deletion
    else:
        print("The specified path does not exist or it is not a directory")



def _print_formatted_field(field_name, obj):
    # Define the fixed width for the field name and the indentation for the object string
    field_width = 30
    indent = " " * 4


    if isinstance(obj, dict):
        print(f"{field_name:<{field_width}}: ")
        try:
            item_str = json.dumps(obj, indent=4)
        except TypeError:
            # Handle non-serializable objects
            item_str = str(obj)
        print(textwrap.indent(item_str, indent))

    # Check if object is a list
    elif isinstance(obj, list):
        # Print the field name with fixed width
        print(f"{field_name:<{field_width}}: ")
        # Print each dictionary in the list on its own row
        for i, item in enumerate(obj):
            if isinstance(item, dict):
                # Convert the dict to a JSON string with indentation
                try:
                    item_str = json.dumps(item, indent=4)
                except TypeError:
                    # Handle non-serializable objects
                    item_str = str(item)
                # If it's not the first item, add an extra newline for separation
                # if i > 0:
                #     print()
                # Print the dictionary with indentation
                print(textwrap.indent(item_str, indent))
            else:
                # For non-dict items in the list, print them with indentation
                print(textwrap.indent(str(item), indent))
    else:
        # Handle other objects (not lists)
        if isinstance(obj, dict):
            obj_str = json.dumps(obj, indent=4)
        else:
            obj_str = str(obj)
        print(f"{field_name:<{field_width}}: {obj_str}")
        #print(textwrap.indent(obj_str, indent))


def print_conversable_agent_state(agent: Agent):
    _print_formatted_field("_name", agent._name)
    if isinstance(agent, ConversableAgent):
        _print_formatted_field("human_input_mode", agent.human_input_mode)
        _print_formatted_field("_code_execution_config", agent._code_execution_config)
        _print_formatted_field("_max_consecutive_auto_reply", agent._max_consecutive_auto_reply)
        _print_formatted_field("_consecutive_auto_reply_counter", agent._consecutive_auto_reply_counter)
        _print_formatted_field("_max_consecutive_auto_reply_dict", agent._max_consecutive_auto_reply_dict)
        _print_formatted_field("_is_termination_msg", agent._is_termination_msg)
        _print_formatted_field("_function_map", agent._function_map)
        _print_formatted_field("_default_auto_reply", agent._default_auto_reply)
        _print_formatted_field("_reply_func_list", agent._reply_func_list)
        _print_formatted_field("reply_at_receive", agent.reply_at_receive)
        _print_formatted_field("_oai_system_message", agent._oai_system_message)
        _print_formatted_field("_oai_messages", agent._oai_messages)
        #print_formatted_field("llm_config",agent.llm_config)



In [88]:
coder_message

{'content': 'Your approach to refining the code snippets is on the right track. You\'ve incorporated the feedback to ensure best practices for data preprocessing, feature engineering, model training, and evaluation. The inclusion of comments and placeholders for unit tests using `pytest` is also commendable. Here are some additional suggestions to further enhance the code:\n\n### Data Loading and Preprocessing\n\n```python\n# filename: /load_and_preprocess.py\n\n# ... (previous code)\n\n# It\'s good practice to encapsulate preprocessing logic within a function\ndef preprocess_data(X, numerical_imputer, categorical_imputer, training=True):\n    """\n    Preprocesses the data by imputing missing values for numerical and categorical columns.\n\n    :param X: DataFrame containing feature data.\n    :param numerical_imputer: Pre-fitted SimpleImputer for numerical data.\n    :param categorical_imputer: Pre-fitted SimpleImputer for categorical data.\n    :param training: Boolean indicating if

In [92]:
! mkdir .code

In [93]:
! chmod +x *

In [94]:
save_code_files(coder_message["content"],work_dir=".code")

Saving code to /load_and_preprocess.py


PermissionError: [Errno 13] Permission denied: '/load_and_preprocess.py'

In [17]:
k = CustomBlueprintML()

In [20]:
import asyncio
loop = asyncio.get_event_loop()
loop.run_until_complete(k.initiate_work(task))


RuntimeError: This event loop is already running

In [95]:
a0 = ConversableAgent(
    name="a0",
    system_message="""You are an experienced Python developer.
    write robust functions.
    """,
    llm_config= llm_config,
    max_consecutive_auto_reply=6,
    human_input_mode="NEVER",
    code_execution_config=False,
)

In [96]:
a1 = ConversableAgent(
    name="a1",
    max_consecutive_auto_reply=3,
    llm_config=False,
    human_input_mode="NEVER",
    code_execution_config=False,
)

In [97]:
critic_agent = ConversableAgent(
    name="critic",
    system_message="You are a python developer. Please give feedback to the other agent and improve the code.",
    max_consecutive_auto_reply=6,
    human_input_mode="NEVER",
    code_execution_config=False,
)

In [98]:
a1.initiate_chat(a0,message="write the python functions for designing a calculator, take inputs as a and b and return the function of a and b")

a1 (to a0):

write the python functions for designing a calculator, take inputs as a and b and return the function of a and b

--------------------------------------------------------------------------------
a0 (to a1):

Certainly! Below is a simple Python calculator that can perform basic arithmetic operations such as addition, subtraction, multiplication, and division. The calculator functions take two inputs, `a` and `b`, and return the result of the operation.

```python
def add(a, b):
    """Return the sum of a and b."""
    return a + b

def subtract(a, b):
    """Return the difference of a and b."""
    return a - b

def multiply(a, b):
    """Return the product of a and b."""
    return a * b

def divide(a, b):
    """Return the quotient of a and b. Raises ZeroDivisionError if b is zero."""
    if b == 0:
        raise ValueError("Cannot divide by zero.")
    return a / b

# Example usage:
try:
    print("Addition of 5 and 3:", add(5, 3))
    print("Subtraction of 5 and 3:", su

In [99]:
message = a0.last_message(a1)

In [100]:
print(message)

{'content': "I'm here to help, but it seems there might be a problem with the messages you're sending—they're arriving empty. If you have a question or need assistance with Python programming or any other topic, please try to send your message again. If there's an issue on your end, please check your connection or device and attempt to resend your query. I'll be here waiting to assist you as soon as I receive your message!", 'role': 'assistant'}


In [102]:
save_code_files(message["content"],".final")

[]